In [ ]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import struct
import warnings

from __future__ import print_function
from matplotlib.mlab import detrend
from obspy import *
from obspy.core import read, UTCDateTime
from obspy.clients.fdsn import Client
from obspy.signal.cross_correlation import xcorr_pick_correction
from obspy.signal.filter import lowpass
from obspy.signal.invsim import cosine_taper 
from scipy import interpolate, signal
from time import *

from lib_seis import smooth
from lib_seis import wf_fft

warnings.filterwarnings('ignore')                  # do not show warnings

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['lines.linewidth'] = 0.5

# Adaptation of CAN_bp.m by Carl Tape
# Coding by: Yongki Aiman, Tomy Gunawan, Angel Ling
# Bandpass filtering the vertical seismogram
# custom functions: smooth, wf_fft

In [ ]:
# get the waveforms
client = Client("IRIS")
t = UTCDateTime("2004-12-26T00:52:23.900")
starttime = UTCDateTime("2004-12-26T00:48:25.600")
endtime = t+(2*3600) # 2 hrs after the earthquake

st = client.get_waveforms("G", "CAN", "*", "BHZ", starttime, endtime, attach_response=True)
print(st)

In [ ]:
can = st.copy()
tr = can[0].data
df = can[0].stats.sampling_rate
times = can[0].times()

# FIX: DO NOT HARD CODE STATION AND CHANNEL
title = 'CAN (BHZ) − starting '+ str(can[0].stats['starttime'])

plt.plot(times,tr, c='b')
plt.xlabel('seconds')
plt.ylabel('counts')
plt.title(title)
plt.show();

In [ ]:
# set up the bandpass filter
T1 = 100   # minimum period: YOU WILL NEED TO CHANGE THIS
T2 = 200   # maximum period: YOU WILL NEED TO CHANGE THIS
f1 = 1/T2
f2 = 1/T1
npoles = 2

can_bp = st.copy()
can_bp.detrend('constant')
can_bp.detrend('linear')
can_bp.taper(0.05)
can_bp.filter('bandpass', freqmin=f1, freqmax=f2, corners=npoles, zerophase=True)

tr1 = can_bp[0].data
plt.plot(times,tr1, c='b')
plt.xlabel('seconds')
plt.ylabel('counts')
plt.title(title)
plt.show();

In [ ]:
# You will need to apply hilbert AND smooth, FOLLOWING NI ET AL. (2005), FIGURE 1

# hilbert doc: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.hilbert.html

# try something like smooth(tr1,npts)
# where npts is the number of points in a sliding window (MUST BE AN ODD INTEGER)

